# 9 - Multilayer Perceptron

**Multi layer perceptron (MLP) is a supplement of feed forward neural network. It consists of three types of layers—the input layer, output layer and hidden layer, as shown in Fig. 3. The input layer receives the input signal to be processed.**

**It is also adapted to regression problems**

# Get dataset

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("Hitters.csv")
data = df.copy()
data.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [6]:
# Check null values
data.isnull().values.any()

True

In [7]:
# Clear null values
data = data.dropna()
data.isnull().values.any()

False

# Create Dummy Variables

In [8]:
dummies = pd.get_dummies(data[["League","Division","NewLeague"]])
dummies.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [9]:
y = data["Salary"]

X_pre = data.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X = pd.concat([X_pre,dummies[["League_N","Division_W","NewLeague_N"]]],axis=1)
X.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League_N,Division_W,NewLeague_N
1,315.0,81.0,7.0,24.0,38.0,39.0,14.0,3449.0,835.0,69.0,321.0,414.0,375.0,632.0,43.0,10.0,1,1,1
2,479.0,130.0,18.0,66.0,72.0,76.0,3.0,1624.0,457.0,63.0,224.0,266.0,263.0,880.0,82.0,14.0,0,1,0
3,496.0,141.0,20.0,65.0,78.0,37.0,11.0,5628.0,1575.0,225.0,828.0,838.0,354.0,200.0,11.0,3.0,1,0,1
4,321.0,87.0,10.0,39.0,42.0,30.0,2.0,396.0,101.0,12.0,48.0,46.0,33.0,805.0,40.0,4.0,1,0,1
5,594.0,169.0,4.0,74.0,51.0,35.0,11.0,4408.0,1133.0,19.0,501.0,336.0,194.0,282.0,421.0,25.0,0,1,0


# Split train and test

In [10]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=33)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(210, 19)
(210,)
(53, 19)
(53,)


# Scale the dataset

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


In [14]:
X_test_scaled[0]

array([ 0.31391358,  0.02281991,  0.95743241,  0.59630231,  0.47546648,
        2.26541197,  2.6470785 ,  2.95765719,  2.70668365,  4.95298222,
        3.34381056,  3.72241088,  3.93726915, -0.93828773, -0.84553287,
       -1.17633279, -0.90971765,  0.87559504, -0.90971765])

# Import the model

In [16]:
from sklearn.neural_network import MLPRegressor
import warnings
warnings.filterwarnings("ignore")

mlpModel = MLPRegressor()
mlpModel.fit(X_train_scaled,y_train)

MLPRegressor()

In [23]:
mlpModel.get_params()

# There is one hidden layer.

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [24]:
mlpModel.n_layers_

3

In [26]:
mlpModel.n_outputs_

1

In [28]:
mlpModel.n_features_in_

19

# Model Tuning

In [29]:
from sklearn.model_selection import GridSearchCV

mlp = MLPRegressor()

mlp_params = {"alpha":[0.01,0.001,0.0001,1e-5,0.000005],
             "hidden_layer_sizes":[(64,64),(128,128,64),(32,64),(64,64)],
             "activation":['tanh','relu','logistic']}

mlp_cv = GridSearchCV(mlp,mlp_params,cv=10)
mlp_cv.fit(X_train_scaled,y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(),
             param_grid={'activation': ['tanh', 'relu', 'logistic'],
                         'alpha': [0.01, 0.001, 0.0001, 1e-05, 5e-06],
                         'hidden_layer_sizes': [(64, 64), (128, 128, 64),
                                                (32, 64), (64, 64)]})

In [30]:
mlp_cv.best_params_

{'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (128, 128, 64)}

# Tuned Model

In [31]:
mlp_tuned = MLPRegressor(alpha=0.0001,hidden_layer_sizes=(128,128,64),activation='relu')
mlp_tuned.fit(X_train_scaled,y_train)

MLPRegressor(hidden_layer_sizes=(128, 128, 64))

In [33]:
# Prediction with tuned model
from sklearn.metrics import mean_squared_error

y_pred_tuned = mlp_tuned.predict(X_test_scaled)
mse_tuned = mean_squared_error(y_test,y_pred_tuned)
rmse_tuned = np.sqrt(mse_tuned)

print(f"MSE Loss Value = {mse_tuned}")
print(f"RMSE Loss Value = {rmse_tuned}")

MSE Loss Value = 83870.0462376644
RMSE Loss Value = 289.60325660749123
